https://www.kaggle.com/c/nlp-getting-started/data

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv("/Users/sdeshpande/Desktop/text_analysis_scripts/nlp-getting-started/train.csv")

In [3]:
train_df.shape

(7613, 5)

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df.fillna("", inplace = True)

In [6]:
train_df['total_text'] = train_df['keyword'] + train_df['location'] + train_df['text']
train_df['total_text'] = train_df['total_text'].apply(lambda x: x.strip())

In [7]:
train_df.head()

,id,keyword,location,text,target,total_text
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...
1,4,,,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada
2,5,,,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...
3,6,,,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or..."
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...


In [8]:
train_df = train_df[['id','total_text','target']]

In [9]:
train_df.set_index('id', inplace = True)
train_df.head()

,total_text,target
id,,
1,Our Deeds are the Reason of this #earthquake M...,1
4,Forest fire near La Ronge Sask. Canada,1
5,All residents asked to 'shelter in place' are ...,1
6,"13,000 people receive #wildfires evacuation or...",1
7,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
#!pip install --upgrade transformers

In [11]:
from simpletransformers.classification import ClassificationModel

In [12]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [13]:
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

In [14]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_tokenize(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_lemmatizer(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join(x))
    return df

In [15]:
train_clean_df = clean_text(train_df, 'total_text', 'total_text_clean')

In [16]:
train_clean_df = train_clean_df[['total_text_clean','target']]
train_clean_df.head()

,total_text_clean,target
id,,
1,deed reason earthquake may allah forgive u,1
4,forest fire near la ronge sask canada,1
5,resident asked shelter place notified officer ...,1
6,people receive wildfire evacuation order calif...,1
7,got sent photo ruby alaska smoke wildfire pour...,1


https://simpletransformers.ai/docs/usage/

In [ ]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 5
model_args.learning_rate = 1e-4

In [19]:
# Create a ClassificationModel
model = ClassificationModel("roberta", "roberta-base", use_cuda=False, args=model_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [20]:
model.train_model(train_clean_df)

Epoch 1 of 3:   0%|          | 0/3 [03:57<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
test_df = pd.read_csv("/Users/sdeshpande/Desktop/text_analysis_scripts/nlp-getting-started/test.csv")
test_df.fillna("", inplace = True)
test_df['total_text'] = test_df['keyword'] + test_df['location'] + test_df['text']
test_df['total_text'] = test_df['total_text'].apply(lambda x: x.strip())

In [ ]:
test_df = test_df[['id','total_text']]
test_df.set_index('id', inplace = True)
test_df.head()

In [ ]:
test_clean_df = clean_text(test_df, 'total_text', 'total_text_clean')

In [ ]:
test_clean_df = test_clean_df[['total_text_clean']]
test_clean_df.head()

In [ ]:
test_clean_df.shape

In [ ]:
prediction_list = test_clean_df.total_text_clean.tolist()

In [ ]:
predictions, raw_outputs = model.predict(prediction_list)

In [ ]:
test_clean_df['predictions'] = predictions

In [ ]:
test_clean_df.head()

In [ ]:
submission_df = test_clean_df[['predictions']]
submission_df.columns = ['target']

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv("submission_file.csv")

In [ ]:
# # Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(eval_df)